In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
from traffic_util import TrafficData
from collections import defaultdict

In [ ]:
# Load dataset as a DataFrame
df = pd.read_pickle('data/dataset_all.pickle')

In [ ]:
# Load TrafficData
data = TrafficData(df)

In [ ]:
print "Number of vehicles %s" %(len(data.df.veh_id.unique()))
print "Number of frames %s" %(len(df.frame_id.unique()))

In [ ]:
# Compute average lane speed
for lane_id, lane_df in df.groupby('lane_id'):
    print "Lane %s average velocity: %s" %(lane_id, lane_df.veh_vel.mean())

In [ ]:
# Process lane change data to extract lane-changes
lane_change_dict = defaultdict(list)
lc_df = data.df.groupby('lane_change').get_group(True)
print "Number of lane changes: %s" %(len(lc_df))
frames_before, frames_after = 60,40
clean_count = 0
for index, row in lc_df.iterrows(): # Iterate over all veh snapshots containing a lanechange
    
    # Skip NaN in from_lane
    if np.isnan(row.from_lane):
        continue
        
    from_lane = int(row.from_lane) #TODO: fix the from_lane in the DataFrame
    lane_id = row.lane_id
    lane_distance = data.get_lane_center(lane_id) - data.get_lane_center(from_lane)
    veh_id = row.veh_id
    lc_frame = row.frame_id
    
    veh_lc_df = data.get_veh_df_between(veh_id,lc_frame - frames_before, lc_frame + frames_after)
    veh_lc_df_pre = veh_lc_df.loc[veh_lc_df.frame_id < lc_frame]
    veh_lc_df_post = veh_lc_df.loc[veh_lc_df.frame_id >= lc_frame]
    if (veh_lc_df_pre.lane_id == from_lane).all() and (veh_lc_df_post.lane_id == lane_id).all():
        # This is a clean lane change
        clean_count = clean_count + 1
        veh_lc_df['lc_frame_index'] = veh_lc_df.frame_id - lc_frame
        veh_lc_df['local_x_normalized'] = (veh_lc_df.local_x - data.get_lane_center(from_lane)) \
            *(1./lane_distance)
        vel_start = veh_lc_df.veh_vel.head(1).squeeze()
        vel_max = max(veh_lc_df.veh_vel.tolist())
        veh_lc_df['veh_vel_normalized'] = (veh_lc_df.veh_vel - vel_start)/vel_max
        
        # Figure out gaps
        frame_start, frame_end = min(veh_lc_df.frame_id),max(veh_lc_df.frame_id)
        lc_snapshot = veh_lc_df[veh_lc_df.frame_id == lc_frame].squeeze()
        veh_id_front = lc_snapshot.veh_front
        veh_id_back = lc_snapshot.veh_back
        if (veh_id_front > 0) and (veh_id_back > 0):
            gap_series =  data.get_diff_series(frame_start, frame_end, veh_id_front, lc_snapshot.veh_id, 'local_y')
            if len(veh_lc_df) == len(gap_series):
                veh_lc_df['gap_length'] = gap_series.tolist()
            else:
                print "veh: %s lc_frame %s veh front: %s veh back: %s frame start: %s frame end: %s len(lc_df) %s len(series) %s" \
                %(lc_snapshot.veh_id, lc_frame, veh_id_front, veh_id_back, frame_start, frame_end, len(veh_lc_df), len(gap_series))
                veh_lc_df['gap_length'] = np.nan
                
        lane_change_dict[(from_lane,lane_id)].append(veh_lc_df)
        
print "Number of clean lane changes %s" %(clean_count)

In [ ]:
# Save to pickle
with open('data/lane_change_dict.pickle','w') as f:
    pickle.dump(lane_change_dict,f)

# Extract lane-keeping samples

In [ ]:
inlane_df_list = list()
for veh_id in data.veh_ids:
    veh_df = data.get_veh_df(veh_id)
    if len(veh_df.lane_id.unique()) == 1 and (veh_df.lane_id.unique()[0] not in [7,8]):
        # Vehicle stays in lane the whole time
        inlane_df_list.append(veh_df)
len(inlane_df_list)

In [ ]:
# Compute lane_dist
lane_dist = dict()
for lane_id in [1,2,3,4,5,6]:
    next_lane = lane_id + 1
    if next_lane > 6:
        next_lane = lane_id -1
    lane_dist[lane_id] = abs(data.get_lane_center(next_lane)-data.get_lane_center(lane_id))

In [ ]:
# Fill local_x_normalized
for veh_df in inlane_df_list:
    lane_id = veh_df.lane_id.unique()[0]
    veh_df['local_x_normalized'] = (veh_df.local_x - data.get_lane_center(lane_id))*(1/lane_dist[lane_id])

In [ ]:
# Convert to observation for HMM
# Extact lateral displacements and velocities
x_seq_list = list()
v_seq_list = list()

for veh_df in inlane_df_list:
    x_seq = np.atleast_2d(veh_df.local_x_normalized.tolist()).T
    v_seq = np.atleast_2d(veh_df.vel_x.tolist()).T
    # Cut the first one since v_seq is nan
    x_seq = x_seq[1:,:]
    v_seq = v_seq[1:,:]
    if (not np.isnan(x_seq).any()) and (not np.isnan(v_seq).any()):
        x_seq_list.append(x_seq)
        v_seq_list.append(v_seq)
        
# Stack the observations
obs_seq_list = list()
for i in range(len(x_seq_list)):
    obs = np.hstack([x_seq_list[i],v_seq_list[i]])
    obs_seq_list.append(obs)

In [ ]:
with open('data/lane_keeping_obs_seq_list.pickle','w') as f:
    pickle.dump(obs_seq_list,f)